<a href="https://colab.research.google.com/github/jhmuller/mlp-covid19/blob/main/mlp_covid19_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
from __future__ import unicode_literals, print_function, division
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [77]:
# need to intall this every time on colab, not sure why but I do it
import os
import sys
import importlib
import warnings
import pdb
import datetime
now = datetime.datetime.now
def warning_on_one_line(message, category, filename, lineno, file=None, line=None):
    return 'line:%s  cat:<%s>  msg:%s\n'% (lineno, category.__name__, message)  

warnings.formatwarning = warning_on_one_line

def global_imports(imports=None, verbosity=0):
  fails = []
  if not imports:
    return
  if not isinstance(imports, list):
    msg = "imports shoule be a list with elements like\m"
    msg += "'foo' or 'foo as bar'"
    warnings.warn(msg)
    return
  #pdb.set_trace()
  for line in imports:
    parts = line.split()
    if len(parts) not in [1,3] or \
          (len(parts) == 3 and parts[1] != 'as'):
      msg = "<{0}> not a valid import line\n"
      msg += "use 'import foo' or 'import foo as bar'"
      warnings.warn(msg)
      continue
    module_name = parts[0]
    asname = parts[0]
    if len(parts) == 3:
      asname = parts[2]
    try:
      module = importlib.import_module(module_name)
      globals()[asname] = module
    except Exception as e:
      print(e)
      print("{0} not installed\Trying to install it".format(module_name))
      try:
        !pip install $module_name
        module = importlib.import_module(module_name)  
        globals()[asname] = module        
      except Exception as e2:
        print(e2)
        fails.append(module_name)
  if fails:
    msg = "Not able to import or install\n{0}".format(fails)
  else:
    msg = "Success. imported {0}".format(imports)
  if verbosity > 0:
    print(msg)
  return fails
    
def is_defined(varnames=None, verbosity=0):
  if isinstance(varnames, str):
    varnames = [varnames]
  for v in varnames:
    try:
      eval(v)
      if verbosity > 0:
        print("'{0}'' is defined")
    except Exception as e:
      warnings.warn("trying to eval '{0}'".format(v))
      print(e)

imports = [ 'fsspec',
           'nltk',
            'pandas as pd',
            'numpy as np']
varnames = ['fsspec', 'pd']
global_imports(imports, verbosity=1)
# check if the imports worked
is_defined([x.split()[-1] for x in imports])

Success. imported ['fsspec', 'nltk', 'pandas as pd', 'numpy as np']


In [78]:
# I am *guessing* this is max_len_text from part 1
max_len_text = 600
MAX_LENGTH = max_len_text

In [79]:
if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  # to mount google drive for the input files
  from google.colab import drive
  drive.mount('/content/gdrive', force_remount=False)  
  dpath = "/content/gdrive/My Drive/mlp_covid19/data/part1_output"  
else:
  print('Not running on CoLab')
  dpath = "./mlp_covid19/data/part1_output"
if not os.path.isdir(dpath):
  msg = "{0} is not a path".format(dpath)
  warnings.warn(msg)
else:
  files = os.listdir(dpath)
  msg = "{0} has {1} files".format(dpath, len(files))
  n2print = 4
  suffix = "..." if len(files) > n2print else ""
  fsizes =[os.path.getsize(os.path.join(dpath, fname)) for fname in files ]
  for i, f in enumerate(files):
    msg += "\n\t'{0}' size: {1} MB  {2} GB".format(f, np.round(fsizes[i]/10**6,2), 
                                             np.round(fsizes[i]/10**9, 3))
  print(msg)

Running on CoLab
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/mlp_covid19/data/part1_output has 4 files
	'pairs_df.csv' size: 123.95 MB  0.124 GB
	'input_lang.parquet' size: 141.22 MB  0.141 GB
	'output_lang.parquet' size: 6.41 MB  0.006 GB
	'pairs.parquet' size: 124.92 MB  0.125 GB


In [80]:
SOS_token = 0
EOS_token = 1
MAX_LENGTH = 20

SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [81]:
# read data in from part 1
import pickle
parq_files = [fname for fname in os.listdir(dpath) if fname.endswith("parquet")]
for pfile in parq_files:
  obj_name = pfile.split(".")[0]
  fpath = os.path.join(dpath, pfile)
  if not os.path.isfile(fpath):
    warnings.warn("{0} is not a filepath".format(fpath))
    continue
  try:
    print("trying to read {0} from {1} {2}".format(pfile, dpath, 
                                                 datetime.datetime.now()))    
    with open(fpath, mode='rb') as file:
      obj = pickle.load(file)
      print("Got it")
      globals()[obj_name] = obj
    print("\nGot it, {0} is a {1}".format(obj_name, type(eval(obj_name))))      
  except Exception as e:
    warnings.warn("*Read Failed*")
    warnings.warn(str(e))    

trying to read input_lang.parquet from /content/gdrive/My Drive/mlp_covid19/data/part1_output 2020-12-12 20:14:28.429750
Got it

Got it, input_lang is a <class '__main__.Lang'>
trying to read output_lang.parquet from /content/gdrive/My Drive/mlp_covid19/data/part1_output 2020-12-12 20:14:29.564535
Got it

Got it, output_lang is a <class '__main__.Lang'>
trying to read pairs.parquet from /content/gdrive/My Drive/mlp_covid19/data/part1_output 2020-12-12 20:14:29.627900
Got it

Got it, pairs is a <class 'list'>


In [82]:
'''
Reusable set of functions to convert a tuple of strings (pair) to tensors
Reference: https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
'''

import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def indexesFromSentence(lang, sentence):
  return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
  indexes = indexesFromSentence(lang, sentence)
  indexes.append(EOS_token)
  return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
  input_tensor = tensorFromSentence(input_lang, pair[0])
  target_tensor = tensorFromSentence(output_lang, pair[1])
  return (input_tensor, target_tensor)

In [83]:
class Encoder(nn.Module):
  def __init__(self, input_size, hidden_size):
    super(Encoder, self).__init__()
    self.hidden_size = hidden_size

    self.embedding = nn.Embedding(input_size, hidden_size)
    self.gru = nn.GRU(hidden_size, hidden_size)

  def init_hidden(self):
    self.hidden = torch.zeros(1, 1, self.hidden_size, device=device)

  def forward(self, input, hidden):
    embedded = self.embedding(input).view(1, 1, -1)
    output = embedded
    output, hidden = self.gru(output, hidden)
    return output, hidden

In [84]:
class AttnDecoder(nn.Module):
  def __init__(self, hidden_size, output_size, dropout=0.1):
    super(AttnDecoder, self).__init__()
    self.hidden_size = hidden_size

    self.embedding = nn.Embedding(output_size, hidden_size)
    self.gru = nn.GRU(hidden_size, hidden_size)
    self.out = nn.Linear(hidden_size, output_size)
    self.softmax = nn.LogSoftmax(dim=1)

  def forward(self, input, hidden):
    output = self.embedding(input).view(1, 1, -1)
    output = F.relu(output)
    output, hidden = self.gru(output, hidden)
    output = self.softmax(self.out(output[0]))
    return output, hidden

  def init_hidden(self):
    return torch.zeros(1, 1, self.hidden_size, device=device)

In [85]:
import time
import math


def asMinutes(s):
  m = math.floor(s / 60)
  s -= m * 60
  return '%dm %ds' % (m, s)


def timeSince(since, percent):
  now = time.time()
  s = now - since
  es = s / (percent)
  rs = es - s
  return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [86]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, 
               learning_rate=0.01, verbosity = 0):
      
  start_dt = time.time()
  plot_losses = []
  print_loss_total = 0  # Reset every print_every
  plot_loss_total = 0  # Reset every plot_every  

  encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
  decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
  training_pairs = [tensorsFromPair(random.choice(pairs))for i in range(n_iters)]
  criterion = nn.NLLLoss()

  if verbosity > 0:
    print("Start iter {0}".format(start_dt))
  losses = []
  for iter in range(1, n_iters + 1):
    if iter% 1000 == 0:
        print(iter,"/",n_iters + 1)
    training_pair = training_pairs[iter - 1]
    input_tensor = training_pair[0]
    target_tensor = training_pair[1]

    loss = train(input_tensor=input_tensor, target_tensor=target_tensor, 
                 encoder=encoder, decoder=decoder, 
                 encoder_optimizer=encoder_optimizer, 
                 decoder_optimizer=decoder_optimizer, 
                 criterion=criterion,
                 verbosity=verbosity) 
    if verbosity > 0:  
      if iter % print_every == 0:
        print_loss_avg = print_loss_total / print_every
        print_loss_total = 0
        print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                      iter, iter / n_iters * 100, 
                                     print_loss_avg))

    if iter % plot_every == 0:
      plot_loss_avg = plot_loss_total / plot_every
      plot_losses.append(plot_loss_avg)
      plot_loss_total = 0
  
  showPlot(plot_losses)      

def  train(input_tensor, target_tensor, encoder, decoder, 
           encoder_optimizer, decoder_optimizer, 
           criterion, max_length=MAX_LENGTH, 
           verbosity=0):
  encoder_hidden = encoder.init_hidden()

  encoder_optimizer.zero_grad()
  decoder_optimizer.zero_grad()

  input_length = input_tensor.size(0)
  target_length = target_tensor.size(0)

  encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
  loss = 0
  for i in range(input_length):
    encoder_output, encoder_hidden = encoder(input_tensor[i], encoder_hidden)
    encoder_outputs[i] = encoder_output[0, 0]
  
  decoder_input = torch.tensor([[SOS_token]], device=device)
  
  decoder_hidden = encoder_hidden

  teacher_forcing_ratio = 0.5  
  use_teacher_forcing = True if random.random < techer_forcing_raio else False
  if use_teacher_forcing:
    # Teacher forcing: Feed the target as the next input
    for di in range(target_length):
      decoder_output, decoder_hidden, decoder_attention = decoder(
           decoder_input, decoder_hidden, encoder_outputs)
      loss += criterion(decoder_output, target_tensor[di])
      decoder_input = target_tensor[di]  # Teacher forcing

  else:
    # Without teacher forcing: use its own predictions as the next input
    for di in range(target_length):
      decoder_output, decoder_hidden, decoder_attention = decoder(
          decoder_input, decoder_hidden, encoder_outputs)
      topv, topi = decoder_output.topk(1)
      decoder_input = topi.squeeze().detach()  # detach from history as input

      loss += criterion(decoder_output, target_tensor[di])
      if decoder_input.item() == EOS_token:
        break

  loss.backward()

  encoder_optimizer.step()
  decoder_optimizer.step()  

  return loss.item / target_length



In [87]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH,
             verbosity=0):
  with torch.no_grad():
    input_tensor = tensorFromSentence(input_lang, sentence)
    input_length = input_tensor.size()[0]
    encoder_hidden = encoder.init_hidden()

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    if verbosity > 0:
      print("encoder outputs.sape {0}".format(encoder_outputs.shape))
      print("input_length= {0} initialing encoder input".format(input_length))
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                  encoder_hidden)
        if verbosity > 0:
          print("{0} type ".format(type(encoder_output)))
        encoder_outputs[ei] += encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

    decoder_hidden = encoder_hidden

    decoded_words = []
    decoder_attentions = torch.zeros(max_length, max_length)

    for di in range(max_length):
      decoder_output, decoder_hidden, decoder_attention = decoder(
          decoder_input, decoder_hidden, encoder_outputs)
      decoder_attentions[di] = decoder_attention.data
      topv, topi = decoder_output.data.topk(1)
      if topi.item() == EOS_token:
        decoded_words.append('<EOS>')
        break
      else:
        decoded_words.append(output_lang.index2word[topi.item()])

        decoder_input = topi.squeeze().detach()

    return decoded_words, decoder_attentions[:di + 1]

In [88]:
def evaluateRandomly(encoder, decoder, n=10):
  for i in range(n):
    pair = random.choice(pairs)
    print('>', pair[0])
    print('=', pair[1])
    output_words, attentions = evaluate(encoder, decoder, pair[0])
    output_sentence = ' '.join(output_words)
    print('<', output_sentence)
    print('')

In [89]:
hidden_size = 300
encoder = Encoder(input_lang.n_words, hidden_size).to(device)
decoder = AttnDecoder(hidden_size, output_lang.n_words, dropout=0.1).to(device)

trainIters(encoder=encoder, decoder=decoder, n_iters=1000, verbosity=2)

Start iter 1607804110.54732


IndexError: ignored